# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Cluster').getOrCreate()
spark

In [8]:
#To load the csv file:
df_cluster = spark.read.csv('hack_data.csv',header=True,inferSchema=True)
df_cluster.show(2)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 2 rows



In [7]:
#To check the number of rows:
df_cluster.count()

334

In [9]:
#To check the numbere of columns:
len(df_cluster.columns)

7

In [12]:
#To print the schema:
df_cluster.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [11]:
#To print the five point summary:
df_cluster.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [14]:
df_cluster.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [16]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [17]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used','Servers_Corrupted',
                                       'Pages_Corrupted','WPM_Typing_Speed'],
                           outputCol='features')

In [19]:
output = assembler.transform(df_cluster)

In [21]:
output.show(1)

+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location|WPM_Typing_Speed|            features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|Slovenia|           72.37|[8.0,391.09,1.0,2...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+--------------------+
only showing top 1 row



In [22]:
from pyspark.ml.feature import StandardScaler

In [36]:
scaler = StandardScaler(inputCol='features', outputCol='Scaler',withMean=False, withStd=True)

In [37]:
scaled_data = scaler.fit(output)

In [38]:
final_data = scaled_data.transform(output)
final_data.show(2)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|              Scaler|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------

In [39]:
from pyspark.ml.clustering import KMeans

In [42]:
kmeans2 = KMeans(featuresCol='Scaler',k=2)
kmeans3 = KMeans(featuresCol='Scaler',k=3)

In [44]:
kmodel2 = kmeans2.fit(final_data)
kmodel3 = kmeans3.fit(final_data)

In [53]:
predict2 = kmodel2.transform(final_data)
predict3 = kmodel3.transform(final_data)

In [86]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

silhoutte2 = evaluator.evaluate(predict2)
silhoutte3 = evaluator.evaluate(predict3)

In [84]:
print("Silhouette score for k=2 = " + str(silhoutte2))

print("Silhouette score for k=3 = " + str(silhoutte3))


Silhouette score for k=2 = 0.6683623593283755
Silhouette score for k=3 = 0.30412315937808737


In [80]:
kmodel3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   79|
|         2|   88|
|         0|  167|
+----------+-----+



In [81]:
kmodel2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [82]:
#As we can see from above results it is clear that for k=2 Count under both the clusters is equal
#silhoutte score is maximum for k=2 it means clusters are very well seperated from each other

In [83]:
#So we can conclude that their were 2 hackers. 